# Import

In [1]:
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

# Data preprocessing

In [2]:
from zipfile import ZipFile

file_name = "/content/drive/MyDrive/datasets/emotions/archive (2).zip"

with ZipFile(file_name, 'r') as zip:
  zip.printdir()
  print("Выгружаю все файлы...")
  zip.extractall()
  print("Готово!")

Streaming output truncated to the last 5000 lines.
train/sad/im3700.png                           2020-12-11 06:08:48         1793
train/sad/im3701.png                           2020-12-11 06:08:48         1701
train/sad/im3702.png                           2020-12-11 06:08:48         1608
train/sad/im3703.png                           2020-12-11 06:08:48         1739
train/sad/im3704.png                           2020-12-11 06:08:48         1680
train/sad/im3705.png                           2020-12-11 06:08:48         1560
train/sad/im3706.png                           2020-12-11 06:08:48         1882
train/sad/im3707.png                           2020-12-11 06:08:48         1680
train/sad/im3708.png                           2020-12-11 06:08:48         1516
train/sad/im3709.png                           2020-12-11 06:08:48         1794
train/sad/im371.png                            2020-12-11 06:08:48         1783
train/sad/im3710.png                           2020-12-11 06:08:48   

In [3]:
shutil.rmtree("/content/test/angry")
shutil.rmtree("/content/train/angry")
shutil.rmtree("/content/test/disgusted")
shutil.rmtree("/content/train/disgusted")

In [4]:
train_dir = "/content/train"
test_dir = "/content/test"

In [5]:
train_datagen = ImageDataGenerator(#rotation_range = 180,
                                   #width_shift_range = 0.1,
                                   #height_shift_range = 0.1,
                                   #horizontal_flip = True,
                                   rescale = 1./255,
                                   #zoom_range = 0.2,
                                   validation_split = 0.2)
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                        validation_split = 0.2)

In [6]:
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(48, 48),
                                                    batch_size=64,
                                                    color_mode="grayscale",
                                                    class_mode="categorical",
                                                    subset="training")

validation_generator = validation_datagen.flow_from_directory(directory = test_dir,
                                                              target_size=(48, 48),
                                                              batch_size=64,
                                                              color_mode="grayscale",
                                                              class_mode="categorical",
                                                              subset="validation")

Found 19423 images belonging to 5 classes.
Found 1219 images belonging to 5 classes.


In [7]:
for x in train_generator.next():
  print(x[0])

[[[0.95294124]
  [0.9843138 ]
  [0.98823535]
  ...
  [0.36862746]
  [0.3019608 ]
  [0.29411766]]

 [[0.97647065]
  [0.9725491 ]
  [0.9803922 ]
  ...
  [0.46274513]
  [0.42352945]
  [0.39607847]]

 [[0.9725491 ]
  [0.9607844 ]
  [0.9686275 ]
  ...
  [0.5529412 ]
  [0.5686275 ]
  [0.57254905]]

 ...

 [[0.2784314 ]
  [0.22352943]
  [0.23137257]
  ...
  [0.19607845]
  [0.16470589]
  [0.13725491]]

 [[0.40784317]
  [0.5019608 ]
  [0.6117647 ]
  ...
  [0.19215688]
  [0.227451  ]
  [0.24705884]]

 [[0.21176472]
  [0.21960786]
  [0.227451  ]
  ...
  [0.28235295]
  [0.2627451 ]
  [0.25490198]]]
[0. 0. 1. 0. 0.]


# Model

In [13]:
model = Sequential()
model.add(Input((48, 48, 1)))
model.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(5, activation="softmax"))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 6, 6, 128)       

In [15]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [16]:
callback = EarlyStopping(monitor="val_loss",
                         mode="min",
                         patience=5)

In [17]:
history = model.fit(x=train_generator,
                    epochs=100,
                    validation_data=validation_generator,
                    callbacks=[callback])

Epoch 1/100
304/304 [==============================] - 7s 22ms/step - loss: 1.4764 - accuracy: 0.3572 - val_loss: 1.2368 - val_accuracy: 0.5103
Epoch 2/100
304/304 [==============================] - 6s 21ms/step - loss: 1.1727 - accuracy: 0.5180 - val_loss: 1.0785 - val_accuracy: 0.5603
Epoch 3/100
304/304 [==============================] - 6s 21ms/step - loss: 1.0417 - accuracy: 0.5816 - val_loss: 1.0044 - val_accuracy: 0.5726
Epoch 4/100
304/304 [==============================] - 6s 21ms/step - loss: 0.9449 - accuracy: 0.6229 - val_loss: 0.9713 - val_accuracy: 0.6120
Epoch 5/100
304/304 [==============================] - 6s 20ms/step - loss: 0.8763 - accuracy: 0.6545 - val_loss: 0.9508 - val_accuracy: 0.6185
Epoch 6/100
304/304 [==============================] - 6s 21ms/step - loss: 0.7915 - accuracy: 0.6909 - val_loss: 0.9560 - val_accuracy: 0.6202
Epoch 7/100
304/304 [==============================] - 6s 21ms/step - loss: 0.7061 - accuracy: 0.7282 - val_loss: 0.9742 - val_accuracy:

In [18]:
model.save("img_to_emotion.h5")